# CP 217: ML4CPS Workshop Week 7

## Week 7 Worksheet:   Perceptron from Scratch

## Perceptron Learning

In this week, we will implement perceptron model from scratch. We again start with generating training data. As you saw in the lectures a perceptron is a linear classifier. Therefore, we will aim to generate linearly separable data.

In [ ]:
import collections
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
def generate_s_shaped_data(gap=3):
    x = np.random.randn(80, 2)

    x[10:20] += np.array([3, 4]) ## keep this datapoints centered at mean (3,4) for (x,y)
    x[20:30] += np.array([0, 8])
    x[30:40] += np.array([3, 12])

    x[40:50] += np.array([gap, 0])
    x[50:60] += np.array([3 + gap, 4])
    x[60:70] += np.array([gap, 8])
    x[70:80] += np.array([3 + gap, 12])

    y = np.hstack([-np.ones(40), np.ones(40)])
    
    d = collections.namedtuple('Dataset', ['x', 'y'])
    d.x = x
    d.y = y
    return d

In [ ]:
d = generate_s_shaped_data(8)
x = d.x
y = d.y

plt.plot(x[y==-1,0], x[y==-1,1], "o")
plt.plot(x[y==1,0], x[y==1,1], "o")

### Perceptron algorithm

Next we will train a binary classifier on this data. For this we’ll use the perceptron algorithm, which
you should recall takes a model of the form
$$\begin{align*}
 s(\mathbf{x}) &= w_0 + \mathbf{w}^T \mathbf{x} \\
 predict(\mathbf{x}) &= \left\{ 
\begin{array}{cc} 
1, & \mbox{if $s(\mathbf{x}) \geq 0$} \\
-1, &  \mbox{otherwise}
\end{array} \right .
\end{align*}$$

Please refer to the lecture notes and the text book for a detailed exposition of the perceptron algorithm and linear classification models in general. Note that we treat $\mathbf{x}$ as a vector above, here it has two elements, one for each of the input dimensions. How many elements must there be in the weight vector $\mathbf{w}$?  

For simplicity, we will use the standard trick to incorporate the bias term $w_0$ into the weights $\mathbf{w}$ by using a basis function $\phi(x_1, x_2) = [1~x_1~x_2]^T$ which adds an extra constant dimension. The model becomes
$$ s(\mathbf{x}) = \mathbf{w}^T \phi(\mathbf{x}) $$
To do this, simply concatenate a column of 1s to the data matrix.

In [ ]:
Phi = np.column_stack([np.ones(x.shape[0]), x])
Phi

Note that Phi now has $3$ columns. In this array, each training instance is a row and each column is a feature. From now on we will use Phi instead of x. Each row represents $\phi(\mathbf{x})$ for a training instance.

### Prediction function

Next, write the prediction function (aka discriminant). This takes as input a data point (a row from Phi, i.e., a vector of 3 numbers) and the model parameters ($\mathbf{w}$) and outputs predicted label $1$ or $-1$. Recall that if $s(\mathbf{x})=0$, the predicted class is $1$.

In [ ]:
def perc_pred(phi, w):
    s= ... # over to you
    return s

Don't forget to test your prediction function with some examples! Note that it's more useful if it can support phi inputs both as vectors (returning a scalar, either +1/-1) and as matrices (returning a vector of +1/-1 values). The latter allows for you to supply a full dataset in one call.

In [ ]:
print(perc_pred([1, 0, 1], [1, 2, 3]))
print(perc_pred(Phi, [1,2,3]))

### Training algorithm

Now for training algorithm which fits the weights, $\mathbf{w}$, to the training data. Recall that this is an online training algorithm, and we are going to iterate through the training examples one by one. Moreover, we are going to do several cycles, called *epochs*, such that we iterate through the entire training set within one epoch. Write a function called *train* which takes the basis data matrix *Phi*, the labels *t* and a number of epochs. This should implement the following pseudo-code:

> initialise weights to zero 

> repeat epoch times

> >   for each x and y pair in the training set

> > >       if model prediction and y differ, make weight update

> return weights

The weight update in the inner loop is $\mathbf{w} \leftarrow \mathbf{w} + y\phi(\mathbf{x})$.
What is the purpose of this update?

Please complete the precition function below.

In [ ]:
def train(data, target, epochs, w=[], eta = 1):
    if len(w) == 0:
        w = np.zeros(data.shape[1])
    for e in range(epochs):
        for i in range(data.shape[0]):
            yhat = ... # over to you
            if yhat != target[i]:
                w = ... # over to you
    return w

Run your training algorithm for 5 epochs to learn the weights

In [ ]:
w_hat = train(Phi, y, 5)
w_hat

### Evaluation

We are going to use the proportion of misclassified cases as the quality measure.

In [ ]:
Accuracy = ... # over to you
print(Accuracy)

Inspect the weights learnt in training. Do these match your intuitions? Plot the decision boundary represented by the weights, $\mathbf{w}^T \phi(\mathbf{x}) = 0$. Solving for $x_2$ as a function of $x_1$ yields $x_2 = -\frac{w_0}{w_2} - \frac{w_1}{w_2} x_1$. Note that you can you *linspace* and *plot* for displaying the line. 

In [ ]:
x1 = np.linspace(-5, 10, 100)
x2 = - w_hat[0] / w_hat[2] - w_hat[1] / w_hat[2] * x1

# plot the decision boundary 
plt.plot(x1, x2)
plt.ylim(-6, 16)
# plot the training data points
plt.plot(x[y==-1,0], x[y==-1,1], "o")
plt.plot(x[y==1,0], x[y==1,1], "o")


***Rerun your training with a larger number of epochs (10, 100, 1000), and evaluate how the accuracy changes.***

### Heldout evaluation

Evaluating on the training data is not a good idea in general, other than for debugging your algorithms. (Can you explain why?) We are going to generate another synthetic data thus essentially creating a fresh *heldout set*. What is the accuracy on this heldout data, and how does this compare to training accuracy?

In [ ]:
d_held = generate_s_shaped_data(8)
x_heldout = d_held.x 
y_heldout = d_held.y


plt.plot(x[y==-1,0], x[y==-1,1], "o")
plt.plot(x[y==1,0], x[y==1,1], "o")

# plot the heldout data points
plt.plot(x_heldout[y_heldout==-1,0], x_heldout[y_heldout==-1,1], "x")
plt.plot(x_heldout[y_heldout==1,0], x_heldout[y_heldout==1,1], "x")


Phi_heldout = np.column_stack([np.ones(x_heldout.shape[0]), x_heldout])

In [ ]:
Accuracy = ... # over to you
print(Accuracy)

Plot decision boundary for heldout data, together with training and heldout datapoints.

In [ ]:
x1 = np.linspace(0, 6, 100)
x2 = - w_hat[0] / w_hat[2] - w_hat[1] / w_hat[2] * x1

# plot the decision boundary 
plt.plot(x1, x2)

# plot the training data points
plt.plot(x[y==-1,0], x[y==-1,1], "o")
plt.plot(x[y==1,0], x[y==1,1], "o")

# plot the heldout data points
plt.plot(x_heldout[y_heldout==-1,0], x_heldout[y_heldout==-1,1], "x")
plt.plot(x_heldout[y_heldout==1,0], x_heldout[y_heldout==1,1], "x")

How well does the decision boundary separate the points in the two classes? Where do you think the decision boundary should go? And how does the boundary change as you train for longer (more epochs)? Plot train and heldout errors as a function of number epochs. Note that careful tuning of the learning rate is needed to get sensible behaviour. Using $\eta = \frac{1}{1+e}$ where $e$ is the epoch number often works well.

In [ ]:
w_hat = []
T = 60
train_error = np.zeros(T)
heldout_error = np.zeros(T)
for e in range(T):
    # here we use a learning rate, which decays with each epoch
    lr = 1./(1+e)
    w_hat = ... # over to you
    train_error[e] = ... # over to you
    heldout_error[e] = ... # over to you

plt.plot(train_error, label = 'Train Error')
plt.plot(heldout_error, label = 'Held-out Error')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Error')

Does the heldout error track the training error closely? Is the model (i.e., weights at a given epoch) on the training set the same as the best model on the heldout set?

Now, let's plot the decision boundary using w_hat

In [ ]:
x1 = np.linspace(2, 10, 100)
print(w_hat)
x2 = - (w_hat[0] / w_hat[2]) - ((w_hat[1] / w_hat[2]) * x1)

# plot the training data points
plt.plot(x[y==-1,0], x[y==-1,1], "o")
plt.plot(x[y==1,0], x[y==1,1], "o")

# plot the heldout data points
plt.plot(x_heldout[y_heldout==-1,0], x_heldout[y_heldout==-1,1], "x")
plt.plot(x_heldout[y_heldout==1,0], x_heldout[y_heldout==1,1], "x")

# plot the decision boundary 
plt.plot(x1, x2)
plt.xlabel('x1')
plt.ylabel('x2')

Now generate training and heldout datasets that are not linearly separable, and investigate what happens to training and heldout errors with increasing number of epochs.

### Classification of SONAR Signals 
The task is to train a network to discriminate between sonar signals bounced off a metal cylinder(M)  and those bounced off a roughly cylindrical rock (R).

The dataset contains 111 patterns obtained by bouncing sonar signals off a metal cylinder (M) and 97 patterns obtained from rocks (R) at various angles and under various conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. 

Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time.

The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder).

You can learn more about this dataset on the UCI Machine Learning repository:
- https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks) 

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataframe = pd.read_csv("sonar_all_data.csv" , header=None)
dataset = dataframe.values

In [ ]:
X = dataset[:,0:60].astype(float)
y = dataset[:,60]
y

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

In [ ]:
encoded_y[encoded_y==0]=-1

In [ ]:
Phi=np.column_stack([np.ones(X.shape[0]), X])
Phi

In [ ]:
Phi.shape

In [ ]:
# split data into train and test sets using Stratified Sampling
from sklearn.model_selection import train_test_split
Phi_train, Phi_test, y_train, y_test = train_test_split(Phi, encoded_y, test_size=0.3, stratify=y)

In [ ]:
w_hat = np.zeros(Phi.shape[1])
T = 100
train_error = np.zeros(T)
heldout_error = np.zeros(T)
for e in range(T):
    lr= 1./(1+e)
    w_hat = train(Phi_train, y_train, 1, w_hat, eta= lr)
    train_error[e] = np.sum(perc_pred(Phi_train, w_hat) != y_train) / float(y_train.shape[0])
    heldout_error[e] = np.sum(perc_pred(Phi_test, w_hat) != y_test) / float(y_test.shape[0])


plt.plot(train_error, label = 'Train Error')
plt.plot(heldout_error, label = 'Held-out Error')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Error')

In [ ]:
Accuracy = np.sum(perc_pred(Phi_test, w_hat) == y_test) / float(y_test.shape[0])
print(Accuracy)

### References
1. Pattern Recognition and Machine Learning, Christopher Bishop, New York, Springer,  2006.
2. https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/
3. Statistical Machine Learning Course Workshop, 2015, University of Melbourne, Australia